<a href="https://colab.research.google.com/github/anikesh-17/House-Price-Prediction/blob/main/House_Price_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
file_path = 'train.csv'  # Update this path
data = pd.read_csv(file_path)

# Handle missing values
data['lot_size'].fillna(data['lot_size'].median(), inplace=True)
data.dropna(subset=['beds', 'baths', 'size', 'price'], inplace=True)

# Convert lot_size_units to a common unit (sqft)
def convert_lot_size_to_sqft(size, unit):
    if unit == 'acre':
        return size * 43560  # 1 acre = 43560 sqft
    return size

data['lot_size'] = data.apply(lambda row: convert_lot_size_to_sqft(row['lot_size'], row['lot_size_units']), axis=1)
data.drop(columns=['lot_size_units', 'size_units'], inplace=True)

# Features and target variable
X = data.drop(columns=['price'])
y = data['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing for numerical and categorical data
numeric_features = ['beds', 'baths', 'size', 'lot_size']
numeric_transformer = StandardScaler()
categorical_features = ['zip_code']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_train_transformed = pipeline.fit_transform(X_train).toarray()
X_test_transformed = pipeline.transform(X_test).toarray()

# Baseline model: Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_transformed, y_train)
y_pred_train = lr_model.predict(X_train_transformed)
y_pred_test = lr_model.predict(X_test_transformed)

print("Linear Regression Performance:")
print(f"Train MAE: {mean_absolute_error(y_train, y_pred_train)}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred_test)}")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train))}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test))}")
print(f"Train R2: {r2_score(y_train, y_pred_train)}")
print(f"Test R2: {r2_score(y_test, y_pred_test)}")

# Neural Network model
def build_model(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

input_shape = X_train_transformed.shape[1]
nn_model = build_model(input_shape)
history = nn_model.fit(X_train_transformed, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

y_pred_train_nn = nn_model.predict(X_train_transformed).flatten()
y_pred_test_nn = nn_model.predict(X_test_transformed).flatten()

print("Neural Network Performance:")
print(f"Train MAE: {mean_absolute_error(y_train, y_pred_train_nn)}")
print(f"Test MAE: {mean_absolute_error(y_test, y_pred_test_nn)}")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_pred_train_nn))}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_test_nn))}")
print(f"Train R2: {r2_score(y_train, y_pred_train_nn)}")
print(f"Test R2: {r2_score(y_test, y_pred_test_nn)}")


Linear Regression Performance:
Train MAE: 253706.60616889715
Test MAE: 235684.08117379114
Train RMSE: 886893.8537983722
Test RMSE: 418362.30086591066
Train R2: 0.23698556677915994
Test R2: 0.46928363514668703
13/13 [==============================] - 0s 2ms/step
Neural Network Performance:
Train MAE: 234784.41001085608
Test MAE: 214525.31779857673
Train RMSE: 902204.8856675683
Test RMSE: 377611.2020127343
Train R2: 0.21041332270699853
Test R2: 0.567638382829805
